# Numpy机器学习基础

这是一份机器学习中应用Numpy的分享，侧重于机器学习中经常会涉及到的Numpy操作；顺带吐槽一下遇到过的坑。

如果需要找内容量适中的教程可以去看菜鸟教程https://www.runoob.com/numpy/numpy-tutorial.html

P.S.对于菜鸟教程的使用方法：
- 最前面的安装和ndarray理解可以再多找找其他资料
- 数组基本操作讲的比较清楚
- 切片索引对于不熟悉的人来说要认真研究
- 广播的原理和常识是一致的，因此可以后面再学
- 迭代数组、位运算没啥用；字节交换也没啥用；IO暂时用不上（除非你需要.npy）
- 字符串批处理、数学&统计函数遇到需求再查就行（一般单词猜的到）
- 数组操作基本只用reshape
- 排序、副本&视图--重在理解
- 矩阵与线性代数：由于代数库可以直接操作数组，所以矩阵用的不多【注意dot函数即可】



In [1]:
import numpy as np

## 常用对象


In [2]:
np.linspace(-1,1,10,retstep=True)

(array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
         0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ]),
 0.2222222222222222)

很多时候要自己建立ndarray的时候，其实最常见的是等差数列，因为画图问题经常要用到一个等距的坐标轴（或者在这个基础上进行坐标变换）。

`np.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)`

常用内容：`np.linspace(start, stop, num=50)`

`retstep=True`会在数组最后附一个间距数值，意义不大（直接能看出来）


要注意的事情有两点：
* 和python惯例不一样，这个是默认上下界都包含在数组里面的
* 拼写别写错，是lin-space没有e

In [3]:
np.arange(1,5,0.5)

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])

当然range也是有的：`np.arange(start, stop, step=1, dtype)`
这个更加接近range，不包含终止值，设定步长

In [4]:
np.logspace(1, 10, num=10, endpoint=True, base=10.0, dtype=None)

array([1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08,
       1.e+09, 1.e+10])

numpy.logspace 函数用于创建一个于**等比**数列。我们不需要先得到linspace再进行变换，可以直接用这个。

`np.logspace(start, stop, num=50, endpoint=True, base=10.0, dtype=None)`

参数和linspace大致相同，主要是多了一个`base`是底数。得到的数组是:$x_0=base^{start},x_n=base^{stop}$，上面的数字是等差的，所以数列是等比的。

## 常用检查方式

说到编程，教程往往给出一个快速打出hello world的流程，但是把实际情况的debug放在高级内容区。但是在基本入门之后最常用的反而是debug操作。

In [5]:
def npCheck(arr,display=False):
    print(f'当前对象的id为：{id(arr)}')#numpy有些是原位操作，要注意这点所以要检查id
    if isinstance(arr,np.matrix):
        print('这是一个np的matrix')
        print(f'矩阵是{arr.shape[0]}*{arr.shape[1]}的')
    elif isinstance(arr,np.ndarray):# 因为matrix也是array的实例
        print('这是一个np的ndarray')
        print(f'维数（轴的数目）是{arr.ndim}')
        print(f'各维度的长度是{arr.shape}')
        print(f'元素的数据类型是{arr.dtype}')
    else:
        print('这似乎不是numpy的对象吖')
    if display:
        print(arr)
    print('-----------')

In [6]:
a = np.arange(24)
b = a.reshape(2,3,4)
npCheck(a)
npCheck(b,True)

当前对象的id为：2254446908560
这是一个np的ndarray
维数（轴的数目）是1
各维度的长度是(24,)
元素的数据类型是int32
-----------
当前对象的id为：2254446907840
这是一个np的ndarray
维数（轴的数目）是3
各维度的长度是(2, 3, 4)
元素的数据类型是int32
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
-----------


In [7]:
c = np.matrix(a.reshape(3,8))
npCheck(c,True)

当前对象的id为：2254446685320
这是一个np的matrix
矩阵是3*8的
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]]
-----------


In [8]:
d = 1
npCheck(d) 

当前对象的id为：140723550868512
这似乎不是numpy的对象吖
-----------


## 副本与视图

Python相比于C的一个好处就是变量命名简便，但是有一个非常经典的代价也来源与此。Numpy在涉及的时候是为了存储大量数据，因此无必要情况不会进行数据复制，这会导致一个大问题。

对于一个变量，在Python中我们是不能通过定义得知它究竟是什么，无论他是一个数据实体还是这个数据的引用，变量名都是随意的标签。这就会导致一个问题：

In [9]:
entity = np.arange(5)
e = entity
print(id(entity)==id(e))

True


这个结果对于Python来说还是比较容易理解的，因为Python的赋值是贴标签，所以两者名字不同但是实质一样

In [10]:
entity = np.arange(5)
print(id(entity))
slice_ = entity[:]
print(id(slice_))
slice_[0] = 2333
print(entity)
print(entity.data,slice_.data)

2254446908800
2254446907680
[2333    1    2    3    4]
<memory at 0x0000020CE77CEA08> <memory at 0x0000020CE77CEAC8>


如果我们对于Numpy的数组进行一个切片，将这个切片赋值给一个变量，虽然切片的id和原来实体不一样(甚至连内存`.data`也不一样），但是切片相当于是原来实体的引用，操作还是会修改原来的数组。

这个一开始不太容易理解，但是考虑到Numpy常用的一个操作就是部分修改数组的数值，而且用的是切片，所以对切片的操作确实应该作用在原数据上。但是这里要注意的一点是，如同再另外做一个temp变量记录，那么要用到copy，否则后面容易把temp当做独立的数据，尽管temp其实还是切片。

这方面有个例子是连续正交化线性回归的操作，对X的正交化求正交的Z时如果不注意的话会修改原有X的数值。

In [11]:
slice2 = slice_.reshape(5,1)
slice_.shape = 1,5
print(slice_)
print(slice2)
print(entity)

[[2333    1    2    3    4]]
[[2333]
 [   1]
 [   2]
 [   3]
 [   4]]
[2333    1    2    3    4]


但是切片不是纯粹等价于原始数据的，改变数值会影响原来的数据，但是rehape不会影响，实际上切片效果和`.view()`函数是一样的，通过reshape或者直接改`.shape`不影响原来的数据，所以切片也可以叫做视图。

总结：
- 换名字：a=b只是变量称呼不同
- 视图：a=b[:],a=b.view()
- 拷贝：a = b[:].copy()